In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras import layers

from matplotlib import pyplot as plt

In [2]:
BATCH_SIZE= 64
AUTOTUNE= tf.data.experimental.AUTOTUNE

In [3]:
(ds_train, ds_test), ds_info= tfds.load(
    'mnist',
    split=['train', 'test'],
    as_supervised= True,
    shuffle_files= True,
    batch_size= BATCH_SIZE,
    with_info= True
)

ds_info

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [4]:
def preprocess(img, label):
    img= tf.cast(img, tf.float32)/255.0

    return img, label

In [5]:
ds_train= ds_train.map(preprocess, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)
ds_test= ds_test.map(preprocess, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)

In [6]:
model = keras.Sequential(
    [
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="model",
)

In [20]:
class CustomFit(keras.Model):
    def __init__(self, model):
        super(CustomFit, self).__init__()
        self.model = model
        self.acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

    # Method saat memanggil compile, di sini kita menginisialisasi optimizer dan loss
    def compile(self, optimizer, loss):
        super(CustomFit, self).compile()
        self.optimizer = optimizer
        self.loss = loss

    # Method untuk fit
    def train_step(self, data):
        # Mengunpack x dan y, x merupakan atribut sedangkan y merupakan label
        x, y = data

        # Di dalam tf.GradientTape() di bawah, apapun yang di-watch() ataupun training=True 
        # Akan disimpan pada self.trainable_variables, yang nantinya akan dilakukan gradient descent padanya
        with tf.GradientTape() as tape:
            # Memprediksi y, di sini kita menyetel training=True karena 
            # Di sini, y_pred akan ditampung pada self.trainable_variables
            y_pred = self.model(x, training=True)

            # Menghitung loss antara y dengan y_pred
            loss = self.loss(y, y_pred)

        # Mendapatkan apa saja variabel training
        training_vars = self.trainable_variables
        print(training_vars)

        # Melakukan gradient descent pada variabel training
        gradients = tape.gradient(loss, training_vars)

        # Step dengan optimizer
        self.optimizer.apply_gradients(zip(gradients, training_vars))
        self.acc_metric.update_state(y, y_pred)

        # Kita akan me-return loss dan akurasi, dan bahkan bisa mereturn apapun sesuka kita
        return {"loss": loss, "accuracy": self.acc_metric.result()}

    # Method saat kita memanggil fungsi evaluate()
    def test_step(self, data):
        x, y = data
        
        y_pred = self.model(x, training=False)
        
        loss = self.loss(y, y_pred)

        # Update metrik
        self.acc_metric.update_state(y, y_pred)

        # Misal kita ingin menambahkan satu variabel lagi pada output dari proses evaluate
        return {"loss": loss, "accuracy": self.acc_metric.result(), "test":69420}

In [21]:
custom_model= CustomFit(model)

In [23]:
custom_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [24]:
custom_model.fit(ds_train, epochs=3)

Epoch 1/3
[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 64) dtype=float32>, <tf.Variable 'conv2d/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 64, 128) dtype=float32>, <tf.Variable 'conv2d_1/bias:0' shape=(128,) dtype=float32>, <tf.Variable 'dense/kernel:0' shape=(100352, 10) dtype=float32>, <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32>]
[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 64) dtype=float32>, <tf.Variable 'conv2d/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 64, 128) dtype=float32>, <tf.Variable 'conv2d_1/bias:0' shape=(128,) dtype=float32>, <tf.Variable 'dense/kernel:0' shape=(100352, 10) dtype=float32>, <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32>]
938/938 [==============================] - 8s 8ms/step - loss: 0.0021 - accuracy: 0.9992
Epoch 2/3
938/938 [==============================] - 8s 8ms/step - loss: 0.0029 - accuracy: 0.9991
Epoch 3/3
938/938 [=============================

Di sini kita bisa melihat apa saja training variable dari model karena kita melakukan print pada method training_step

In [25]:
custom_model.evaluate(ds_test)

157/157 [==============================] - 1s 8ms/step - loss: 0.0991 - accuracy: 0.9871 - test: 69420.0000


[0.9871000051498413, 5.587910436588572e-07, 69420]